In [1]:
import pandas as pd
import gspread as gs
import warnings
import sys
warnings.filterwarnings('ignore')

In [2]:
gc = gs.service_account(filename="C:/Users/leste/OneDrive/Desktop/Python files/mypythonprogrammes/Mydatafiles/service_account.json")

In [3]:
google_spreadsheet = gc.open_by_url("https://docs.google.com/spreadsheets/d/1cM1pMxHZof65AeVIUJphyYZze3Ew0myrzqlmg9tvQdg/edit#gid=1044390699")

<font face = "Times New Roman" size = 5 color = "orange"> Creating DataFrames </font>

In [4]:

def extract_sheets():
    token_ids = []
    sheets = google_spreadsheet.worksheets()
    for sheet in sheets:
        token_ids.append(sheet)
    return token_ids
    
extract_sheets()
    

[<Worksheet 'AAPL' id:0>,
 <Worksheet 'MSFT' id:177394609>,
 <Worksheet 'GOOG' id:1261668908>,
 <Worksheet 'AMZN' id:1631741944>,
 <Worksheet 'TSLA' id:1799150376>,
 <Worksheet 'NVDA' id:669265052>,
 <Worksheet 'META' id:1044390699>]

In [5]:
df_dict = {}

def declare_dataframes():
    
    for i in extract_sheets():
        df = pd.DataFrame(i.get_all_records())
        df_dict.update({f"{i.title}_stock" : df})
      
        
   
declare_dataframes()

<font face = "Times New Roman" size = 5 color = "yellow">Parsing dates</font>

In [6]:


def to_datetime(dataframes):
    try:
        
        for data_frame in dataframes:
            if len(df_dict) != len(token_ids):
                sys.exit("The no. of google sheets does not match the count of dataframe objects")
            else:
                data_frame.set_index("Date",inplace = True)
                data_frame.index = pd.to_datetime(data_frame.index , infer_datetime_format= True)
                warnings.filterwarnings('ignore')
                
    
    except KeyError:
        print("FUNCTION EXECUTED SUCCESSFULLY")
        print(type(data_frame.index))

to_datetime(df_dict.values())